In [1]:
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_columns',None)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()

import matplotlib.pyplot as plt 

plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import seaborn as sns 
from tqdm import tqdm
tqdm.pandas()

import datetime
import ast
import math
import gc

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=7, progress_bar=True)

from sklearn.preprocessing import MinMaxScaler

def say(comment):
  os.system(f'say "{comment}"')

# warming 제거
import warnings
warnings.filterwarnings('ignore')
from branca.colormap import linear

# notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것
%matplotlib inline       

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus']= False

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
s2p = pd.read_pickle('../RawData2/s2p/0211_1353_skill2pgm_1차완성_ver2.pkl')

In [5]:
s2p['쇼호스트명'].apply(lambda x: len(x) if x is not None else 0).max()

5

In [6]:
s2p.head()

,방송편성번호,인입콜,year,month,day,hour,PGM명,상품구분,쇼호스트명,날씨,기온,강수량,방송시작일시,weekday,duration,sorted_판매상품명,sorted_상품대분류명,sorted_상품중분류명,sorted_상품소분류명,sorted_상품세분류명,sorted_브랜드명,comb_판매상품명,comb_상품대분류명,comb_상품중분류명,comb_상품소분류명,comb_상품세분류명,comb_브랜드명,상품판매가,판매가평균,midcat_order_score,midcat_etc_score,midcat_order_score_mean,midcat_etc_score_mean,midcat_order_score_max,midcat_etc_score_max,prod_num,한정표현구분,한정표현_num,주말/공휴일,sh_num,sh_grade,sh_grade_num,시차변수,소비지수,month_sin,month_cos,hour_sin,hour_cos,day_sin,day_cos,approx_in_show_call,approx_out_show_call
0,202001015001,33,2020.0,1,1,6,(D19_P)소담 진공쌀통(김홍근10_15),DATA,"[엄지혜, 이경은]",구름 많음,-4.0,0.0,2020-01-01 06:00:00,수,40,[소담 진공쌀통],[주방],[보관용품],[쌀독],[쌀통],[소담],[소담 진공쌀통],[주방],[보관용품],[쌀독],[쌀통],[소담],[99000],99000.0,[2],[2],2.0,2.0,2,2,1,[없음],0,0,2.0,D,1.0,NaN,104.8,0.5,0.866025,1.000000,6.123234e-17,0.201299,0.97953,19.208955,13.791045
1,202001015002,80,2020.0,1,1,6,(D19_N)코지마 3D 목어깨 마사지기(박정우09_23),DATA,"[신성애, 장민철]",구름 많음,-4.0,0.0,2020-01-01 06:40:00,수,60,[코지마 3D 목어깨 마사지기 트위스터 CMN-100WL],[생활/건강],[건강],[안마/마사지기],[마사지기],[코지마],[코지마 3D 목어깨 마사지기 트위스터 CMN-100WL],[생활/건강],[건강],[안마/마사지기],[마사지기],[코지마],[99000],99000.0,[3],[3],3.0,3.0,3,3,1,[없음],0,1,2.0,D,1.0,NaN,104.8,0.5,0.866025,1.000000,6.123234e-17,0.201299,0.97953,53.698630,26.301370
2,202001015003,203,2020.0,1,1,7,(D19_Y)헤드골프 하이브리드 패딩수트(주재광11_28),DATA,"[심용수, 유지수]",구름 많음,-4.0,0.0,2020-01-01 07:40:00,수,60,"[헤드골프 남성 하이브리드 패딩수트 상하의 2종, 헤드골프 여성 하이브리드 패딩수트...","[스포츠/레져, 스포츠/레져]","[골프용품, 골프용품]","[골프웨어, 골프웨어]","[상하의세트(남성), 상하의세트(여성)]","[헤드, 헤드]","[헤드골프 남성 하이브리드 패딩수트 상하의 2종, 헤드골프 여성 하이브리드 패딩수트...","[스포츠/레져, 스포츠/레져]","[골프용품, 골프용품]","[골프웨어, 골프웨어]","[상하의세트(남성), 상하의세트(여성)]","[헤드, 헤드]","[99000,99000]",99000.0,"[2, 2]","[4, 4]",2.0,4.0,2,4,2,[없음],0,1,2.0,C,2.0,NaN,104.8,0.5,0.866025,0.965926,-2.588190e-01,0.201299,0.97953,150.409326,52.590674
3,202001015004,188,2020.0,1,1,8,(D18_B)진휘 때필링(임흥준10_18),DATA,[이찬석],흐림,0.0,0.0,2020-01-01 08:40:00,수,60,"[진휘 때필링 1 세트, 진휘 때필링 세트]","[이미용, 이미용]","[이미용품, 이미용품]","[바디용품, 바디용품]","[필링, 필링]","[진휘, 진휘]","[진휘 때필링 세트, 진휘 때필링 1 세트]","[이미용, 이미용]","[이미용품, 이미용품]","[바디용품, 바디용품]","[필링, 필링]","[진휘, 진휘]","[49900,39000]",44450.0,"[2, 2]","[2, 2]",2.0,2.0,2,2,2,[없음],0,1,1.0,S,5.0,NaN,104.8,0.5,0.866025,0.866025,-5.000000e-01,0.201299,0.97953,51.700000,136.300000
4,202001015005,72,2020.0,1,1,9,(D19_N)아이스크림 홈런(양선10_18),DATA,[신진아],흐림,0.0,0.0,2020-01-01 09:40:00,수,60,[[렌탈]아이스크림 홈런],[서비스],[회원권/이용권],[렌탈(일반)],[학습기기],[아이스크림홈런],[[렌탈]아이스크림 홈런],[서비스],[회원권/이용권],[렌탈(일반)],[학습기기],[아이스크림홈런],[2332500],2332500.0,[3],[3],3.0,3.0,3,3,1,[없음],0,1,1.0,D,1.0,NaN,104.8,0.5,0.866025,0.707107,-7.071068e-01,0.201299,0.97953,9.649485,62.350515


In [26]:
all_ref = {}

In [27]:
# pgm ref 생성
pgm_ref = s2p.PGM명.drop_duplicates().reset_index(drop=True)

# 쇼호스트 ref 생성
sh_ref = []
for sh_li in s2p.쇼호스트명.drop_duplicates():
  if sh_li is not None:
    for sh in sh_li:
      if sh not in sh_ref:
        sh_ref.append(sh)

# 날씨 ref 생성
weather_ref = s2p.날씨.drop_duplicates().reset_index(drop=True)

# 요일 ref 생성
weekday_ref = s2p.weekday.drop_duplicates().reset_index(drop=True)

# 브랜드명 ref 생성
brand_ref = []
for brand_li in s2p.sorted_브랜드명.drop_duplicates():
  if brand_li is not None:
    for brand in brand_li:
      if brand not in brand_ref:
        brand_ref.append(brand)


In [29]:
all_ref['pgm_ref'] = pgm_ref
all_ref['showhost_ref'] = sh_list

<function dict.values>

In [30]:
import joblib
# joblib.dump(all_ref, 'ref.pkl')